In [1]:
!pip install geopandas
!pip install contextily

  Using cached contextily-1.6.2-py3-none-any.whl.metadata (2.9 kB)
  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached mercantile-1.2.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
Using cached contextily-1.6.2-py3-none-any.whl (17 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached mercantile-1.2.1-py3-none-any.whl (14 kB)
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)


In [2]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
   
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

def wait_for_job(job_id: str) -> str:
    return job_status_for(job_id)

In [3]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import pandas as pd
import geopandas as gpd
import backoff
import shapely
import glob
import os

In [4]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [5]:
iso3 = "HND"  ##ARG URY
file_path = f"/projects/my-public-bucket/AOIs/vero_1deg_tileindex/tileindex_{iso3}.csv"
tileindex_df = pd.read_csv(file_path)
# Display the first few rows of the DataFrame
#print(tileindex_df.head())

In [6]:
json_files = tileindex_df['s3path']
AOIs = json_files
print(len(AOIs))
aoi_name = os.path.basename(AOIs[0]).split('.')[0]
aoi_name

22


'tile_num_16275'

In [29]:
AOIs[1]

'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_16276.geojson'

In [30]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L2B" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover,pai,fhd_normal",#"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result



Thanks, I'll run the GEDI L4A subsetter for tile_num_16275!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16276!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16455!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16456!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16457!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16635!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16636!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16637!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16638!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16814!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16815!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16816!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16817!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16994!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16995!!
Thanks, I'll run the GEDI L4A subsetter for tile_num_16996!!
Thanks, I'll run the GED

In [14]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

#PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L2B" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover,pai,fhd_normal",#"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result


Thanks, I'll run the GEDI L2A subsetter for tile_num_15735!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15736!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15913!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15914!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15915!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15916!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_15917!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16093!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16094!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16095!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16096!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16097!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16273!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16274!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16275!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_16276!!
Thanks, I'll run the GED

In [15]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

#PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L2B" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover,pai,fhd_normal",#"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

        
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result


Thanks, I'll run the GEDI L2B subsetter for tile_num_15735!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_15736!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_15913!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_15914!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_15915!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_15916!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_15917!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_16093!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_16094!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_16095!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_16096!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_16097!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_16273!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_16274!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_16275!!
Thanks, I'll run the GEDI L2B subsetter for tile_num_16276!!
Thanks, I'll run the GED

In [6]:
##L4A
## if necessary ## re-run failed subsets
#BRA
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_27090.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_27111.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_27291.geojson'
#CHL
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17937.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17944.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_18117.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_18304.geojson'
#COL
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17727.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17897.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_18267.geojson'
#ECU
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_15749.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_15929.geojson'
#SUR
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22224.geojson'
#VEN

#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_27090.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_27111.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22080.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_16276.geojson'
each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22620.geojson'

inputs = dict(
    aoi=each_aoi,
    doi="L4A",
    lat="lat_lowestmode",
    lon="lon_lowestmode",
    beams="all",
    columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
    query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
    limit = 75_000,
    #temporal="-",
    output=f"{os.path.basename(each_aoi).split('.')[0]}_L4A.gpkg"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset",
    version="0.7.0",
    queue="maap-dps-worker-32gb",
    username="leitoldv",
    **inputs
)
inputs
job_id = result.id
job_id or result

'24652a3c-ccd2-4e49-b1c7-3120dbb9872c'

In [7]:
#PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

## if necessary ## re-run failed subsets
#BRA
each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22620.geojson'

inputs = dict(
    aoi=each_aoi,
    doi="L2A",
    lat="lat_lowestmode",
    lon="lon_lowestmode",
    beams="all",
    columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
    #columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
    #columns="all",
    query="quality_flag == 1",
    limit = 75_000,
    #temporal="-",
    output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset",
    version="0.7.0",
    queue="maap-dps-worker-32gb",
    username="leitoldv",
    **inputs
)
inputs
job_id = result.id
job_id or result

'cdd4b045-d990-4221-946d-3f989268fd34'

In [ ]:
##L2B
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_16111.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_17915.geojson'

#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_23865.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_23869.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_24040.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_24213.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_25481.geojson'

#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_15916.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_15914.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_15914.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_15914.geojson'

inputs = dict(
    aoi=each_aoi,
    doi="L2B",
    lat="geolocation/lat_lowestmode",
    lon="geolocation/lon_lowestmode",
    beams="all",
    columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover,pai,fhd_normal",#"+",".join(variables),
    query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
    limit = 75_000,
    #temporal="-",
    output=f"{os.path.basename(each_aoi).split('.')[0]}_L2B.gpkg"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset",
    version="0.7.0",
    queue="maap-dps-worker-32gb",
    username="leitoldv",
    **inputs
)
inputs
job_id = result.id
job_id or result

In [35]:
## L4A

In [6]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


1755


In [7]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.7.0/gedi-subset/2024/09/08"  # Can now set name
out_dir = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L4A_tiles"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L4A.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
#            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
            print(f"Copying {fname}...")
            subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

Copying tile_num_22620_L4A.gpkg...


In [65]:
## L2A

In [8]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


934


In [9]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.7.0/gedi-subset/2024/09/08"  # Can now set name
out_dir = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L2A.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
#            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
            print(f"Copying {fname}...")
            subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

Copying tile_num_22080_L2A.gpkg...
Copying tile_num_22259_L2A.gpkg...
Copying tile_num_23160_L2A.gpkg...
Copying tile_num_22620_L2A.gpkg...


In [13]:
## L2B

In [25]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2B_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


191


In [26]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.7.0/gedi-subset/2024/08/30"  # Can now set name
out_dir = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2B_tiles"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L2B.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
#            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
            print(f"Copying {fname}...")
            subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

Copying tile_num_18613_L2B.gpkg...
Copying tile_num_18433_L2B.gpkg...
Copying tile_num_18432_L2B.gpkg...
Copying tile_num_18252_L2B.gpkg...
Copying tile_num_18612_L2B.gpkg...
Copying tile_num_17721_L2B.gpkg...
Copying tile_num_17901_L2B.gpkg...
Copying tile_num_18263_L2B.gpkg...
Copying tile_num_17902_L2B.gpkg...
Copying tile_num_18081_L2B.gpkg...
Copying tile_num_18443_L2B.gpkg...
Copying tile_num_18083_L2B.gpkg...
Copying tile_num_18082_L2B.gpkg...
Copying tile_num_17722_L2B.gpkg...
Copying tile_num_17903_L2B.gpkg...
Copying tile_num_17541_L2B.gpkg...
Copying tile_num_18261_L2B.gpkg...
Copying tile_num_17723_L2B.gpkg...
Copying tile_num_18442_L2B.gpkg...
Copying tile_num_18441_L2B.gpkg...
Copying tile_num_17542_L2B.gpkg...
Copying tile_num_18262_L2B.gpkg...
Copying tile_num_17362_L2B.gpkg...
Copying tile_num_16097_L2B.gpkg...
Copying tile_num_16095_L2B.gpkg...
Copying tile_num_16096_L2B.gpkg...
Copying tile_num_15913_L2B.gpkg...
Copying tile_num_16093_L2B.gpkg...
Copying tile_num_159